In [8]:
# load in data files
def load_dat_file(file_name, n_components):  
    """
    Loads in .dat files as lists. 
    
    Inputs: 
        
    """
    new_lines = []
    with open(file_name) as input_file:
        for line in input_file:
            new_line = [x.strip() for x in line.split('|')]
            
            if len(new_line) == n_components:
                new_lines.append(new_line)
    return new_lines

checkin_data = load_dat_file('./raw_data/checkins.dat', 6)
ratings_data = load_dat_file('./raw_data/ratings.dat', 3)
socialgraph_data = load_dat_file('./raw_data/socialgraph.dat', 2)
user_data = load_dat_file('./raw_data/users.dat', 3)
venues_data = load_dat_file('./raw_data/venues.dat', 3)